In [1]:
import random

In [2]:
class Individual:
    
    def __init__(self, chromosome_length):
        self.chromosome = None
        self.set_chromosome(chromosome_length)
        self.fitness = None
    
    # Initialize with random chromosome
    def set_chromosome(self, chromosome_length):
        self.chromosome = []
        for _ in xrange(chromosome_length):
            if random.uniform(0, 1) < 0.5:
                self.chromosome.append(0)
            else:
                self.chromosome.append(1)

In [3]:
class Population:
    
    def __init__(self, population_size, chromosome_length=None):
        # Set None list for crossover or mutation to set individual
        self.individuals = [None] * population_size
        if chromosome_length is not None:
            self.set_individuals(population_size, chromosome_length)
        self.population_fitness = None
    
    # Initialize individuals in population
    def set_individuals(self, population_size, chromosome_length):
        self.individuals = []
        for _ in xrange(population_size):
            self.individuals.append(Individual(chromosome_length))
    
    # The offset of the individual you want, sorted by fitness
    # 0 is the strongest, len(population)-1 is the weakest
    def get_fittest(self, offset):
        self.individuals.sort(key=lambda x: x.fitness, reverse=True)
        return self.individuals[offset]
    
    def shuffle(self):
        random.shuffle(self.individuals)

In [4]:
class GeneticAlgorithm:
    
    def __init__(self, population_size, mutation_rate, crossover_rate, elitism_count):
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.elitism_count = elitism_count
        
    def init_population(self, chromosome_length):
        # Initialize population
        return Population(self.population_size, chromosome_length)
    
    def calc_fitness(self, individual):
        correct_genes = 0
        
        # Loop over individual's genes
        for i in range(len(individual.chromosome)):
            # Add one fitness point for each '1' found
            if individual.chromosome[i] == 1:
                correct_genes += 1
        
        # Calculate fitness
        fitness = correct_genes / float(len(individual.chromosome))
        
        # Store fitness
        individual.fitness = fitness
        
        return fitness
    
    def eval_population(self, population):
        population_fitness = 0
        
        for individual in population.individuals:
            population_fitness += self.calc_fitness(individual)
        
        population.population_fitness = population_fitness
    
    def termination_condition_met(self, population):
        for individual in population.individuals:
            if individual.fitness == 1:
                return True
        return False
    
    def select_parent(self, population):
        individuals = population.individuals
        
        population_fitness = population.population_fitness
        roulette_wheel_position = random.uniform(0, 1) * population_fitness
        
        spin_wheel = 0
        for individual in individuals:
            spin_wheel += individual.fitness
            if spin_wheel >= roulette_wheel_position:
                return individual
        return individuals[len(population_size) - 1]
    
    def crossover_population(self, population):
        # Create new population
        new_population = Population(len(population.individuals))
        
        # Loop over current population by fitness
        for population_index in range(len(population.individuals)):
            parent1 = population.get_fittest(population_index)
            
            # Apply crossover to this individual?
            if self.crossover_rate > random.uniform(0, 1) and population_index >= self.elitism_count:
                # Initialize offspring
                offspring = Individual(len(parent1.chromosome))
                
                # Find second parent
                parent2 = self.select_parent(population)
                
                # loop over genome
                for gene_index in range(len(parent1.chromosome)):
                    # Use half of parent1's gene and half of parent2's genes
                    if random.uniform(0, 1) < 0.5:
                        offspring.chromosome[gene_index] = parent1.chromosome[gene_index]
                    else:
                        offspring.chromosome[gene_index] = parent2.chromosome[gene_index]
                        
                new_population.individuals[population_index] = offspring
            else:
                new_population.individuals[population_index] = parent1
        
        return new_population
        
    def mutate_population(self, population):
        # Initialize new population
        new_population = Population(self.population_size)
        
        # Loop over current population by fitness
        for population_index in range(len(population.individuals)):
            individual = population.get_fittest(population_index)
            
            # Loop over individual's genes
            for gene_index in range(len(individual.chromosome)):
                # Skip mutation if this is an elite individual
                if population_index > self.elitism_count:
                    # Does this gene need mutation?
                    if self.mutation_rate > random.uniform(0, 1):
                        # Get new gene
                        new_gene = 1
                        if individual.chromosome[gene_index] == 1:
                            new_gene = 0
                        # Mutate gene
                        individual.chromosome[gene_index] = new_gene
            
            # Add individual to population
            new_population.individuals[population_index] = individual
            
        return new_population

In [5]:
# Create GA object
ga = GeneticAlgorithm(100, 0.001, 0.95, 2)

# Intiialize population
population = ga.init_population(50)

# Evaluate population
ga.eval_population(population)

# Keep track of current generation
generation = 1

while ga.termination_condition_met(population) is False:
    # Print fittest individual from population
    print "Best solution: " + ''.join([str(x) for x in population.get_fittest(0).chromosome])
    
    # Apply crossover
    population = ga.crossover_population(population)
    
    # Apply mutation
    population = ga.mutate_population(population)
    
    # Evaluate population
    ga.eval_population(population)
    
    # Increment the current generation
    generation += 1

print 'Found solution in ' + str(generation) + ' generations'
print 'Best solution: ' + ''.join([str(x) for x in population.get_fittest(0).chromosome])

Best solution: 01101111010111101101101111111111101011111010000101
Best solution: 11011000111111001100111000001111111111111111011111
Best solution: 11011000111111001100111000001111111111111111011111
Best solution: 11011000111111001100111000001111111111111111011111
Best solution: 11011000111111001100111000001111111111111111011111
Best solution: 11011000111111001100111000001111111111111111011111
Best solution: 11101101101110011111100011100111111111011011011111
Best solution: 11101101101110011111100011100111111111011011011111
Best solution: 11101101101110011111100011100111111111011011011111
Best solution: 11101101101110011111100011100111111111011011011111
Best solution: 11101101101110011111100011100111111111011011011111
Best solution: 11100011101110000111110111111111111101101111101111
Best solution: 11100011101110000111110111111111111101101111101111
Best solution: 11110111111110110111111011111011101111001111101110
Best solution: 11110111111110110111111011111011101111001111101110
Best solut